![QuantConnect Logo](https://cdn.quantconnect.com/web/i/qc_notebook_logo_rev0.png)
## Welcome to The QuantConnect Research Page
#### Refer to this page for documentation https://www.quantconnect.com/docs#Introduction-to-Jupyter
#### Contribute to this template file https://github.com/QuantConnect/Lean/blob/master/Jupyter/BasicQuantBookTemplate.ipynb

## QuantBook Basics

### Start QuantBook
- Add the references and imports
- Create a QuantBook instance

In [ ]:
%matplotlib inline
# Imports
from clr import AddReference
AddReference("System")
AddReference("QuantConnect.Common")
AddReference("QuantConnect.Jupyter")
AddReference("QuantConnect.Indicators")
from System import *
from QuantConnect import *
from QuantConnect.Data.Custom import *
from QuantConnect.Data.Market import TradeBar, QuoteBar
from QuantConnect.Jupyter import *
from QuantConnect.Indicators import *
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd

# Create an instance
qb = QuantBook()

### Selecting Asset Data
Checkout the QuantConnect [docs](https://www.quantconnect.com/docs#Initializing-Algorithms-Selecting-Asset-Data) to learn how to select asset data.

In [ ]:
spy = qb.AddEquity("SPY")
eur = qb.AddForex("EURUSD")
btc = qb.AddCrypto("BTCUSD")
fxv = qb.AddData[FxcmVolume]("EURUSD_Vol", Resolution.Hour)

### Historical Data Requests

We can use the QuantConnect API to make Historical Data Requests. The data will be presented as multi-index pandas.DataFrame where the first index is the Symbol.

For more information, please follow the [link](https://www.quantconnect.com/docs#Historical-Data-Historical-Data-Requests).

In [ ]:
# Gets historical data from the subscribed assets, the last 360 datapoints with daily resolution
h1 = qb.History(qb.Securities.Keys, 360, Resolution.Daily)

In [ ]:
# Plot closing prices from "SPY" 
h1.loc["SPY"]["close"].plot()

In [ ]:
# Gets historical data from the subscribed assets, from the last 30 days with daily resolution
h2 = qb.History(qb.Securities.Keys, timedelta(360), Resolution.Daily)

In [ ]:
# Plot high prices from "EURUSD" 
h2.loc["EURUSD"]["high"].plot()

In [ ]:
# Gets historical data from the subscribed assets, between two dates with daily resolution
h3 = qb.History([btc.Symbol], datetime(2014,1,1), datetime.now(), Resolution.Daily)

In [ ]:
# Plot closing prices from "BTCUSD" 
h3.loc["BTCUSD"]["close"].plot()

In [ ]:
# Only fetchs historical data from a desired symbol
h4 = qb.History([spy.Symbol], 360, Resolution.Daily)
# or qb.History(["SPY"], 360, Resolution.Daily)

In [ ]:
# Only fetchs historical data from a desired symbol
h5 = qb.History([eur.Symbol], timedelta(360), Resolution.Daily)
# or qb.History(["EURUSD"], timedelta(30), Resolution.Daily)

In [ ]:
# Fetchs custom data
h6 = qb.History([fxv.Symbol], timedelta(360))
h6.loc[fxv.Symbol.Value]["volume"].plot()

### Historical Options Data Requests
- Select the option data
- Sets the filter, otherwise the default will be used SetFilter(-1, 1, timedelta(0), timedelta(35))
- Get the OptionHistory, an object that has information about the historical options data

In [ ]:
goog = qb.AddOption("GOOG")
goog.SetFilter(-2, 2, timedelta(0), timedelta(180))

In [ ]:
option_history = qb.GetOptionHistory(goog.Symbol, datetime(2017, 1, 4))
print (option_history.GetStrikes())
print (option_history.GetExpiryDates())
h7 = option_history.GetAllData()

### Historical Future Data Requests
- Select the future data
- Sets the filter, otherwise the default will be used SetFilter(timedelta(0), timedelta(35))
- Get the FutureHistory, an object that has information about the historical future data

In [ ]:
es = qb.AddFuture("ES")
es.SetFilter(timedelta(0), timedelta(180))

In [ ]:
future_history = qb.GetFutureHistory(es.Symbol, datetime(2017, 1, 4))
print (future_history.GetExpiryDates())
h7 = future_history.GetAllData()

### Get Fundamental Data

- *GetFundamental([symbol], selector, start_date = datetime(1998,1,1), end_date = datetime.now())*

We will get a pandas.DataFrame with fundamental data.

In [ ]:
data = qb.GetFundamental(["AAPL","AIG","BAC","GOOG","IBM"], "ValuationRatios.PERatio")
data

### Indicators

We can easily get the indicator of a given symbol with QuantBook. 

For all indicators, please checkout QuantConnect Indicators [Reference Table](https://www.quantconnect.com/docs#Indicators-Reference-Table)

In [ ]:
# Example with BB, it is a datapoint indicator
# Define the indicator
bb = BollingerBands(30, 2)

# Gets historical data of indicator
bbdf = qb.Indicator(bb, "SPY", 360, Resolution.Daily)

# drop undesired fields
bbdf = bbdf.drop('standarddeviation', 1)

# Plot
bbdf.plot()

In [ ]:
# For EURUSD
bbdf = qb.Indicator(bb, "EURUSD", 360, Resolution.Daily)
bbdf = bbdf.drop('standarddeviation', 1)
bbdf.plot()

In [ ]:
# Example with ADX, it is a bar indicator
adx = AverageDirectionalIndex("adx", 14)
adxdf = qb.Indicator(adx, "SPY", 360, Resolution.Daily)
adxdf.plot()

In [ ]:
# For EURUSD
adxdf = qb.Indicator(adx, "EURUSD", 360, Resolution.Daily)
adxdf.plot()

In [ ]:
# Example with ADO, it is a tradebar indicator (requires volume in its calculation)
ado = AccumulationDistributionOscillator("ado", 5, 30)
adodf = qb.Indicator(ado, "SPY", 360, Resolution.Daily)
adodf.plot()

In [ ]:
# For EURUSD. 
# Uncomment to check that this SHOULD fail, since Forex is data type is not TradeBar.
# adodf = qb.Indicator(ado, "EURUSD", 360, Resolution.Daily)
# adodf.plot()

In [ ]:
# SMA cross:
symbol = "EURUSD"
# Get History 
hist = qb.History([symbol], 500, Resolution.Daily)
# Get the fast moving average
fast = qb.Indicator(SimpleMovingAverage(50), symbol, 500, Resolution.Daily)
# Get the fast moving average
slow = qb.Indicator(SimpleMovingAverage(200), symbol, 500, Resolution.Daily)

# Remove undesired columns and rename others 
fast = fast.drop('rollingsum', 1).rename(columns={'simplemovingaverage': 'fast'})
slow = slow.drop('rollingsum', 1).rename(columns={'simplemovingaverage': 'slow'})

# Concatenate the information and plot 
df = pd.concat([hist.loc[symbol]["close"], fast, slow], axis=1).dropna(axis=0)
df.plot()

In [ ]:
# Get indicator defining a lookback period in terms of timedelta
ema1 = qb.Indicator(ExponentialMovingAverage(50), "SPY", timedelta(100), Resolution.Daily)
# Get indicator defining a start and end date
ema2 = qb.Indicator(ExponentialMovingAverage(50), "SPY", datetime(2016,1,1), datetime(2016,10,1), Resolution.Daily)

ema = pd.concat([ema1, ema2], axis=1)
ema.plot()

In [ ]:
rsi = RelativeStrengthIndex(14)

# Selects which field we want to use in our indicator (default is Field.Close)
rsihi = qb.Indicator(rsi, "SPY", 360, Resolution.Daily, Field.High)
rsilo = qb.Indicator(rsi, "SPY", 360, Resolution.Daily, Field.Low)
rsihi = rsihi.rename(columns={'relativestrengthindex': 'high'})
rsilo = rsilo.rename(columns={'relativestrengthindex': 'low'})
rsi = pd.concat([rsihi['high'], rsilo['low']], axis=1)
rsi.plot()